# 1. Importo librerias

In [1]:
from binance.client import Client
from datetime import datetime
from backtesting import Backtest, Strategy
from backtesting.lib import plot_heatmaps

import multiprocessing
import pandas as pd


/Users/nelsonrios/opt/anaconda3/lib/python3.8/site-packages/backtesting/_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

# 2. Configuro ambiente y traigo datos historicos

In [2]:
## necesario por cuestiones de optimizacion de performance de la simulacion
multiprocessing.set_start_method('fork', force=True)

In [2]:
#TODO: mover a variables de entorno
api_key = ''
secret_key = ''

client = Client(api_key, secret_key)

In [3]:
# bars = client.get_historical_klines('BTCUSDT', Client.KLINE_INTERVAL_5MINUTE, start_str='2021-03-30 00:00:00', end_str='2021-06-11 00:00:00')
bars = client.get_historical_klines('ETHUSDT', Client.KLINE_INTERVAL_4HOUR, '6 months ago')

def cast_string_to_float(data):
    if type(data) == str:
        casted_data = float(data)
    else:
        casted_data = datetime.fromtimestamp(data/1000.0)
    return casted_data

# delete unwanted data - just keep date, open, high, low, close
formated_bars = []
for line in bars:
    del line[6:]
    # cast string numbers to integer
    formated_bars.append(list(map(cast_string_to_float, line)))

#  store data in a dataframe
df = pd.DataFrame(formated_bars, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
df.set_index('Date', inplace=True)
# Converting the index as date
df.index = pd.to_datetime(df.index)
df

,Open,High,Low,Close,Volume
Date,,,,,
2021-02-12 01:00:00,1764.56,1773.00,1739.88,1768.28,118850.48721
2021-02-12 05:00:00,1768.38,1782.20,1749.50,1775.61,99182.72272
2021-02-12 09:00:00,1775.61,1802.00,1750.89,1789.03,193728.10339
2021-02-12 13:00:00,1789.20,1851.10,1787.08,1840.36,253823.30175
2021-02-12 17:00:00,1840.36,1864.88,1827.99,1840.26,130190.54405
...,...,...,...,...,...
2021-08-11 09:00:00,3214.39,3274.00,3206.25,3245.22,132981.30073
2021-08-11 13:00:00,3245.23,3257.00,3220.00,3249.65,88080.30947
2021-08-11 17:00:00,3249.66,3264.80,3150.00,3160.50,127875.05511


# 3. Armo una estrategia de velas Heikin Ashi


In [156]:
close_serie

2021-07-29 13:27:00         NaN
2021-07-29 13:28:00    0.000000
2021-07-29 13:29:00    0.000115
2021-07-29 13:30:00    0.000000
2021-07-29 13:31:00    0.000000
                         ...   
2021-08-02 16:22:00    0.000000
2021-08-02 16:23:00    0.000000
2021-08-02 16:24:00    0.000000
2021-08-02 16:25:00    0.000804
2021-08-02 16:26:00    0.000589
Name: Close, Length: 5940, dtype: float64

## 3.1 Estrategia

In [29]:
from talib import EMA
from backtesting.lib import crossover

class TrendFollow_HA(Strategy):
    SLOW_LENGTH = 29
    FAST_LENGTH = 6

    def calculate_heikin_ashi_ohlc(self):
        open = pd.Series(self.data.Open)
        high = pd.Series(self.data.High)
        low = pd.Series(self.data.Low)
        close = pd.Series(self.data.Close)
        frame = {
            'Open': open,
            'High': high,
            'Low': low,
            'Close': close
        }

        df = pd.DataFrame(frame)

        df['HA_Close']=(df['Open']+ df['High']+ df['Low']+df['Close'])/4

        idx = df.index.name
        df.reset_index(inplace=True)

        for i in range(0, len(df)):
            if i == 0:
                df.at[i, 'HA_Open'] = (df.at[i, 'Open'] + df.at[i, 'Close']) / 2
            else:
                df.at[i, 'HA_Open'] = (df.at[i - 1, 'HA_Open'] + df.at[i - 1, 'HA_Close']) / 2

        if idx:
            df.set_index(idx, inplace=True)

        df['HA_High']=df[['HA_Open','HA_Close','High']].max(axis=1)
        df['HA_Low']=df[['HA_Open','HA_Close','Low']].min(axis=1)
        df.drop(columns=['Open', 'High', 'Low', 'Close'], inplace=True)

        # df['Color'] = 1
        # df.loc[df['HA_Open'] > df['HA_Close'], 'Color'] = 0

        # df.loc[df['HA_Low'] == df['HA_Open']] = True
        return (df['HA_High'] + df['HA_Low']) / 2


    def init(self):
        hl2 = self.calculate_heikin_ashi_ohlc()
        self.slow_ema = self.I(EMA, hl2, timeperiod=self.SLOW_LENGTH)
        self.fast_ema = self.I(EMA, hl2, timeperiod=self.FAST_LENGTH)

    def next(self):
        if crossover(self.fast_ema, self.slow_ema):
            self.buy()
        elif crossover(self.slow_ema, self.fast_ema):
            self.sell()


## 3.2 Resultados

In [30]:
bt = Backtest(df, TrendFollow_HA, cash=1000000, commission=.0004, exclusive_orders=True)
stats = bt.run()
stats

Start                     2021-02-09 01:00:00
End                       2021-08-08 21:00:00
Duration                    180 days 20:00:00
Exposure Time [%]                   96.593002
Equity Final [$]               2846942.111376
Equity Peak [$]                2987246.261376
Return [%]                         184.694211
Buy & Hold Return [%]               68.794622
Return (Ann.) [%]                  724.685599
Volatility (Ann.) [%]              988.735612
Sharpe Ratio                         0.732942
Sortino Ratio                       11.941463
Calmar Ratio                         22.84082
Max. Drawdown [%]                  -31.727653
Avg. Drawdown [%]                   -6.224234
Max. Drawdown Duration       51 days 12:00:00
Avg. Drawdown Duration        4 days 06:00:00
# Trades                                   36
Win Rate [%]                        41.666667
Best Trade [%]                      55.419738
Worst Trade [%]                    -10.300031
Avg. Trade [%]                    

In [31]:
bt.plot()

## 3.3 Optimizacion

In [25]:
stats, heatmap = bt.optimize(FAST_LENGTH = range(3,30), SLOW_LENGTH = range(5, 50), maximize='Equity Final [$]', return_heatmap=True, constraint=lambda param: param.FAST_LENGTH < param.SLOW_LENGTH)

/Users/nelsonrios/opt/anaconda3/lib/python3.8/site-packages/backtesting/backtesting.py:1472: UserWarning: Searching for best of 890 configurations.
  output = _optimize_grid()


In [26]:
 plot_heatmaps(heatmap, agg='mean')


Column(id='4306', ...)

In [27]:
stats

Start                     2021-02-09 01:00:00
End                       2021-08-08 21:00:00
Duration                    180 days 20:00:00
Exposure Time [%]                   96.593002
Equity Final [$]               2846942.111376
Equity Peak [$]                2987246.261376
Return [%]                         184.694211
Buy & Hold Return [%]               68.794622
Return (Ann.) [%]                  724.685599
Volatility (Ann.) [%]              988.735612
Sharpe Ratio                         0.732942
Sortino Ratio                       11.941463
Calmar Ratio                         22.84082
Max. Drawdown [%]                  -31.727653
Avg. Drawdown [%]                   -6.224234
Max. Drawdown Duration       51 days 12:00:00
Avg. Drawdown Duration        4 days 06:00:00
# Trades                                   36
Win Rate [%]                        41.666667
Best Trade [%]                      55.419738
Worst Trade [%]                    -10.300031
Avg. Trade [%]                    

In [28]:
stats._strategy

<Strategy TrendFollow_HA(FAST_LENGTH=6,SLOW_LENGTH=29)>

In [ ]:
# 4. Comparo contra version sin heikin ashi

In [39]:
from talib import EMA
from backtesting.lib import crossover

class TrendFollow(Strategy):
    SLOW_LENGTH = 13
    FAST_LENGTH = 3


    def init(self):
        self.slow_ema = self.I(EMA, self.data.Close, timeperiod=self.SLOW_LENGTH)
        self.fast_ema = self.I(EMA, self.data.Close, timeperiod=self.FAST_LENGTH)

    def next(self):
        if crossover(self.fast_ema, self.slow_ema):
            self.buy()
        elif crossover(self.slow_ema, self.fast_ema):
            self.sell()


In [40]:
bt = Backtest(df, TrendFollow, cash=1000000, commission=.0004, exclusive_orders=True)
stats = bt.run()
stats

Start                     2021-02-09 01:00:00
End                       2021-08-08 21:00:00
Duration                    180 days 20:00:00
Exposure Time [%]                   98.618785
Equity Final [$]                3386808.26532
Equity Peak [$]                3489617.973864
Return [%]                         238.680827
Buy & Hold Return [%]               68.794622
Return (Ann.) [%]                 1070.475376
Volatility (Ann.) [%]             1308.967582
Sharpe Ratio                         0.817801
Sortino Ratio                       19.207395
Calmar Ratio                        38.393754
Max. Drawdown [%]                  -27.881498
Avg. Drawdown [%]                   -6.309931
Max. Drawdown Duration       61 days 12:00:00
Avg. Drawdown Duration        3 days 19:00:00
# Trades                                   73
Win Rate [%]                        39.726027
Best Trade [%]                      62.651299
Worst Trade [%]                      -7.79861
Avg. Trade [%]                    

In [41]:
bt.plot()

In [35]:
stats, heatmap = bt.optimize(FAST_LENGTH = range(3,30), SLOW_LENGTH = range(5, 50), maximize='Equity Final [$]', return_heatmap=True, constraint=lambda param: param.FAST_LENGTH < param.SLOW_LENGTH)

/Users/nelsonrios/opt/anaconda3/lib/python3.8/site-packages/backtesting/backtesting.py:1472: UserWarning: Searching for best of 890 configurations.
  output = _optimize_grid()


In [36]:
 plot_heatmaps(heatmap, agg='mean')


Column(id='5136', ...)

In [37]:
stats

Start                     2021-02-09 01:00:00
End                       2021-08-08 21:00:00
Duration                    180 days 20:00:00
Exposure Time [%]                   98.618785
Equity Final [$]                3386808.26532
Equity Peak [$]                3489617.973864
Return [%]                         238.680827
Buy & Hold Return [%]               68.794622
Return (Ann.) [%]                 1070.475376
Volatility (Ann.) [%]             1308.967582
Sharpe Ratio                         0.817801
Sortino Ratio                       19.207395
Calmar Ratio                        38.393754
Max. Drawdown [%]                  -27.881498
Avg. Drawdown [%]                   -6.309931
Max. Drawdown Duration       61 days 12:00:00
Avg. Drawdown Duration        3 days 19:00:00
# Trades                                   73
Win Rate [%]                        39.726027
Best Trade [%]                      62.651299
Worst Trade [%]                      -7.79861
Avg. Trade [%]                    

In [38]:
stats._strategy

<Strategy TrendFollow(FAST_LENGTH=3,SLOW_LENGTH=13)>

In [42]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,556,15,34,1797.118560,1804.55,4131.880640,0.004135,2021-02-11 13:00:00,2021-02-14 17:00:00,3 days 04:00:00
1,-556,34,40,1803.828180,1824.55,-11521.331920,-0.011488,2021-02-14 17:00:00,2021-02-15 17:00:00,1 days 00:00:00
2,543,40,41,1825.279820,1779.33,-24950.752260,-0.025174,2021-02-15 17:00:00,2021-02-15 21:00:00,0 days 04:00:00
3,-544,41,42,1778.618268,1822.16,-23686.702208,-0.024481,2021-02-15 21:00:00,2021-02-16 01:00:00,0 days 04:00:00
4,517,42,44,1822.888864,1783.12,-20560.502688,-0.021816,2021-02-16 01:00:00,2021-02-16 09:00:00,0 days 08:00:00
...,...,...,...,...,...,...,...,...,...,...
68,1408,973,1008,1876.000100,2172.81,417908.339200,0.158214,2021-07-21 05:00:00,2021-07-27 01:00:00,5 days 20:00:00
69,-1408,1008,1010,2171.940876,2278.35,-149824.046592,-0.048993,2021-07-27 01:00:00,2021-07-27 09:00:00,0 days 08:00:00
70,1276,1010,1051,2279.261340,2493.10,272858.130160,0.093819,2021-07-27 09:00:00,2021-08-03 05:00:00,6 days 20:00:00
71,-1277,1051,1059,2492.102760,2672.57,-230456.665480,-0.072416,2021-08-03 05:00:00,2021-08-04 13:00:00,1 days 08:00:00


# Pruebo en otros timeframes


In [88]:
# bars = client.get_historical_klines('BTCUSDT', Client.KLINE_INTERVAL_5MINUTE, start_str='2021-03-30 00:00:00', end_str='2021-06-11 00:00:00')
bars = client.get_historical_klines('ETHUSDT', Client.KLINE_INTERVAL_1HOUR, '6 months ago')

def cast_string_to_float(data):
    if type(data) == str:
        casted_data = float(data)
    else:
        casted_data = datetime.fromtimestamp(data/1000.0)
    return casted_data

# delete unwanted data - just keep date, open, high, low, close
formated_bars = []
for line in bars:
    del line[6:]
    # cast string numbers to integer
    formated_bars.append(list(map(cast_string_to_float, line)))

#  store data in a dataframe
df = pd.DataFrame(formated_bars, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
df.set_index('Date', inplace=True)
# Converting the index as date
df.index = pd.to_datetime(df.index)
df

,Open,High,Low,Close,Volume
Date,,,,,
2021-02-09 15:00:00,1739.85,1756.01,1735.21,1750.44,30412.08820
2021-02-09 16:00:00,1750.45,1772.56,1743.03,1772.56,34791.91277
2021-02-09 17:00:00,1772.56,1791.58,1762.65,1763.38,53716.29559
2021-02-09 18:00:00,1763.63,1772.00,1757.94,1766.15,30783.80711
2021-02-09 19:00:00,1766.15,1766.16,1750.21,1765.11,22947.15340
...,...,...,...,...,...
2021-08-09 13:00:00,3166.62,3174.78,3128.08,3157.32,42093.58636
2021-08-09 14:00:00,3157.33,3170.96,3137.44,3138.97,26215.09930
2021-08-09 15:00:00,3138.98,3186.00,3138.97,3162.63,34964.10217


In [93]:
from talib import EMA
from backtesting.lib import crossover

class TrendFollow(Strategy):
    SLOW_LENGTH = 48
    FAST_LENGTH = 12


    def init(self):
        self.slow_ema = self.I(EMA, self.data.Close, timeperiod=self.SLOW_LENGTH)
        self.fast_ema = self.I(EMA, self.data.Close, timeperiod=self.FAST_LENGTH)

    def next(self):
        if crossover(self.fast_ema, self.slow_ema):
            self.buy()
        elif crossover(self.slow_ema, self.fast_ema):
            self.sell()

In [94]:
bt = Backtest(df, TrendFollow, cash=1000000, commission=.0004, exclusive_orders=True)
stats = bt.run()
stats

Start                     2021-02-09 15:00:00
End                       2021-08-09 17:00:00
Duration                    181 days 02:00:00
Exposure Time [%]                   97.211982
Equity Final [$]               3040334.955308
Equity Peak [$]                3554412.471736
Return [%]                         204.033496
Buy & Hold Return [%]               76.694431
Return (Ann.) [%]                  830.028548
Volatility (Ann.) [%]              942.815943
Sharpe Ratio                         0.880372
Sortino Ratio                       15.933991
Calmar Ratio                        28.146041
Max. Drawdown [%]                  -29.490064
Avg. Drawdown [%]                     -5.3192
Max. Drawdown Duration       45 days 22:00:00
Avg. Drawdown Duration        2 days 09:00:00
# Trades                                   90
Win Rate [%]                        31.111111
Best Trade [%]                      68.321425
Worst Trade [%]                     -7.494706
Avg. Trade [%]                    

In [95]:
bt.plot()

In [96]:
stats, heatmap = bt.optimize(FAST_LENGTH = range(10,50), SLOW_LENGTH = range(20, 50), maximize='Return [%]', return_heatmap=True, constraint=lambda param: param.FAST_LENGTH < param.SLOW_LENGTH)

/Users/nelsonrios/opt/anaconda3/lib/python3.8/site-packages/backtesting/backtesting.py:1472: UserWarning: Searching for best of 735 configurations.
  output = _optimize_grid()


In [97]:
 plot_heatmaps(heatmap, agg='mean')


Column(id='10464', ...)

In [98]:
stats

Start                     2021-02-09 15:00:00
End                       2021-08-09 17:00:00
Duration                    181 days 02:00:00
Exposure Time [%]                   98.940092
Equity Final [$]               3594994.615656
Equity Peak [$]                 4143197.89292
Return [%]                         259.499462
Buy & Hold Return [%]               76.694431
Return (Ann.) [%]                 1201.516792
Volatility (Ann.) [%]             1325.857972
Sharpe Ratio                         0.906218
Sortino Ratio                       23.350278
Calmar Ratio                        45.858787
Max. Drawdown [%]                  -26.200362
Avg. Drawdown [%]                   -4.189547
Max. Drawdown Duration       43 days 12:00:00
Avg. Drawdown Duration        1 days 23:00:00
# Trades                                   95
Win Rate [%]                        35.789474
Best Trade [%]                      66.801092
Worst Trade [%]                     -6.782459
Avg. Trade [%]                    

In [99]:
stats._strategy

<Strategy TrendFollow(FAST_LENGTH=25,SLOW_LENGTH=26)>

In [100]:
stats, heatmap = bt.optimize(FAST_LENGTH = range(10,50), SLOW_LENGTH = range(20, 50), maximize='Max. Drawdown [%]', return_heatmap=True, constraint=lambda param: param.FAST_LENGTH < param.SLOW_LENGTH)

/Users/nelsonrios/opt/anaconda3/lib/python3.8/site-packages/backtesting/backtesting.py:1472: UserWarning: Searching for best of 735 configurations.
  output = _optimize_grid()


In [101]:
 plot_heatmaps(heatmap, agg='mean')


Column(id='10580', ...)

In [102]:
stats

Start                     2021-02-09 15:00:00
End                       2021-08-09 17:00:00
Duration                    181 days 02:00:00
Exposure Time [%]                   98.894009
Equity Final [$]               2740759.121352
Equity Peak [$]                3158624.455764
Return [%]                         174.075912
Buy & Hold Return [%]               76.694431
Return (Ann.) [%]                  655.348933
Volatility (Ann.) [%]              755.230487
Sharpe Ratio                         0.867747
Sortino Ratio                       12.352777
Calmar Ratio                        27.109981
Max. Drawdown [%]                  -24.173714
Avg. Drawdown [%]                   -5.404457
Max. Drawdown Duration       33 days 16:00:00
Avg. Drawdown Duration        2 days 10:00:00
# Trades                                   87
Win Rate [%]                        33.333333
Best Trade [%]                      67.009699
Worst Trade [%]                     -7.476255
Avg. Trade [%]                    

In [103]:
stats._strategy

<Strategy TrendFollow(FAST_LENGTH=19,SLOW_LENGTH=36)>

## Pruebo data historica


In [75]:
# bars = client.get_historical_klines('BTCUSDT', Client.KLINE_INTERVAL_5MINUTE, start_str='2021-03-30 00:00:00', end_str='2021-06-11 00:00:00')
bars = client.get_historical_klines('ETHUSDT', Client.KLINE_INTERVAL_1HOUR, '12 months ago')

def cast_string_to_float(data):
    if type(data) == str:
        casted_data = float(data)
    else:
        casted_data = datetime.fromtimestamp(data/1000.0)
    return casted_data

# delete unwanted data - just keep date, open, high, low, close
formated_bars = []
for line in bars:
    del line[6:]
    # cast string numbers to integer
    formated_bars.append(list(map(cast_string_to_float, line)))

#  store data in a dataframe
df = pd.DataFrame(formated_bars, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
df.set_index('Date', inplace=True)
# Converting the index as date
df.index = pd.to_datetime(df.index)
df

,Open,High,Low,Close,Volume
Date,,,,,
2020-08-09 15:00:00,385.09,390.15,385.02,390.00,17108.98825
2020-08-09 16:00:00,390.00,391.13,388.83,389.48,18512.27831
2020-08-09 17:00:00,389.48,392.95,389.48,392.86,16940.56808
2020-08-09 18:00:00,392.85,394.30,388.01,389.17,22173.99663
2020-08-09 19:00:00,389.17,391.27,387.20,388.81,18398.28317
...,...,...,...,...,...
2021-08-09 13:00:00,3166.62,3174.78,3128.08,3157.32,42093.58636
2021-08-09 14:00:00,3157.33,3170.96,3137.44,3138.97,26215.09930
2021-08-09 15:00:00,3138.98,3186.00,3138.97,3162.63,34964.10217


In [78]:
from talib import EMA
from backtesting.lib import crossover

class TrendFollow(Strategy):
    SLOW_LENGTH = 26
    FAST_LENGTH = 25


    def init(self):
        self.slow_ema = self.I(EMA, self.data.Close, timeperiod=self.SLOW_LENGTH)
        self.fast_ema = self.I(EMA, self.data.Close, timeperiod=self.FAST_LENGTH)

    def next(self):
        if crossover(self.fast_ema, self.slow_ema):
            self.buy()
        elif crossover(self.slow_ema, self.fast_ema):
            self.sell()

In [77]:
bt = Backtest(df, TrendFollow, cash=1000000, commission=.0004, exclusive_orders=True)
stats = bt.run()
stats

Start                     2020-08-09 15:00:00
End                       2021-08-09 17:00:00
Duration                    365 days 02:00:00
Exposure Time [%]                   99.542909
Equity Final [$]              10708767.888412
Equity Peak [$]               12342266.886196
Return [%]                         970.876789
Buy & Hold Return [%]              699.338462
Return (Ann.) [%]                  963.961736
Volatility (Ann.) [%]             1238.288871
Sharpe Ratio                         0.778463
Sortino Ratio                       17.822652
Calmar Ratio                        28.895997
Max. Drawdown [%]                  -33.359698
Avg. Drawdown [%]                   -4.525349
Max. Drawdown Duration       85 days 19:00:00
Avg. Drawdown Duration        2 days 13:00:00
# Trades                                  194
Win Rate [%]                        34.536082
Best Trade [%]                      66.801092
Worst Trade [%]                     -6.782459
Avg. Trade [%]                    

In [79]:
bt = Backtest(df, TrendFollow, cash=1000000, commission=.0004, exclusive_orders=True)
stats = bt.run()
stats

Start                     2020-08-09 15:00:00
End                       2021-08-09 17:00:00
Duration                    365 days 02:00:00
Exposure Time [%]                   99.531482
Equity Final [$]              10929742.893256
Equity Peak [$]               12596967.758396
Return [%]                         992.974289
Buy & Hold Return [%]              699.338462
Return (Ann.) [%]                  985.855946
Volatility (Ann.) [%]              1265.97552
Sharpe Ratio                         0.778732
Sortino Ratio                       18.227848
Calmar Ratio                        29.552273
Max. Drawdown [%]                  -33.359733
Avg. Drawdown [%]                   -4.501582
Max. Drawdown Duration       85 days 14:00:00
Avg. Drawdown Duration        2 days 12:00:00
# Trades                                  192
Win Rate [%]                        34.895833
Best Trade [%]                      66.801092
Worst Trade [%]                     -6.782459
Avg. Trade [%]                    

In [80]:
stats, heatmap = bt.optimize(FAST_LENGTH = range(12,50), SLOW_LENGTH = range(15, 50), maximize='Return [%]', return_heatmap=True, constraint=lambda param: param.FAST_LENGTH < param.SLOW_LENGTH)

/Users/nelsonrios/opt/anaconda3/lib/python3.8/site-packages/backtesting/backtesting.py:1472: UserWarning: Searching for best of 700 configurations.
  output = _optimize_grid()


In [81]:
 plot_heatmaps(heatmap, agg='mean')

Column(id='8804', ...)

In [82]:
stats

Start                     2020-08-09 15:00:00
End                       2021-08-09 17:00:00
Duration                    365 days 02:00:00
Exposure Time [%]                   99.097246
Equity Final [$]              11194127.669472
Equity Peak [$]               13087734.151352
Return [%]                        1019.412767
Buy & Hold Return [%]              699.338462
Return (Ann.) [%]                  1012.04961
Volatility (Ann.) [%]             1283.027283
Sharpe Ratio                         0.788798
Sortino Ratio                       18.779239
Calmar Ratio                        34.309154
Max. Drawdown [%]                  -29.497947
Avg. Drawdown [%]                   -4.890784
Max. Drawdown Duration       74 days 04:00:00
Avg. Drawdown Duration        2 days 16:00:00
# Trades                                  187
Win Rate [%]                        32.620321
Best Trade [%]                      68.321425
Worst Trade [%]                     -7.494706
Avg. Trade [%]                    

In [83]:
stats._strategy

<Strategy TrendFollow(FAST_LENGTH=12,SLOW_LENGTH=48)>

In [84]:
stats, heatmap = bt.optimize(FAST_LENGTH = range(12,45), SLOW_LENGTH = range(15, 45), maximize='Max. Drawdown [%]', return_heatmap=True, constraint=lambda param: param.FAST_LENGTH < param.SLOW_LENGTH)

/Users/nelsonrios/opt/anaconda3/lib/python3.8/site-packages/backtesting/backtesting.py:1472: UserWarning: Searching for best of 525 configurations.
  output = _optimize_grid()


In [85]:
 plot_heatmaps(heatmap, agg='mean')

Column(id='8920', ...)

In [86]:
stats

Start                     2020-08-09 15:00:00
End                       2021-08-09 17:00:00
Duration                    365 days 02:00:00
Exposure Time [%]                   99.097246
Equity Final [$]               9668716.706808
Equity Peak [$]                11289484.88528
Return [%]                         866.871671
Buy & Hold Return [%]              699.338462
Return (Ann.) [%]                   860.89641
Volatility (Ann.) [%]              1116.68699
Sharpe Ratio                         0.770938
Sortino Ratio                       15.712413
Calmar Ratio                        33.623156
Max. Drawdown [%]                  -25.604271
Avg. Drawdown [%]                   -5.137717
Max. Drawdown Duration       74 days 01:00:00
Avg. Drawdown Duration        2 days 17:00:00
# Trades                                  179
Win Rate [%]                        34.636872
Best Trade [%]                      67.009699
Worst Trade [%]                     -7.476255
Avg. Trade [%]                    

In [87]:
stats._strategy

<Strategy TrendFollow(FAST_LENGTH=16,SLOW_LENGTH=41)>

# Conclusion


In [4]:
# bars = client.get_historical_klines('BTCUSDT', Client.KLINE_INTERVAL_5MINUTE, start_str='2021-03-30 00:00:00', end_str='2021-06-11 00:00:00')
bars = client.get_historical_klines('ETHUSDT', Client.KLINE_INTERVAL_1HOUR, '2 months ago')

def cast_string_to_float(data):
    if type(data) == str:
        casted_data = float(data)
    else:
        casted_data = datetime.fromtimestamp(data/1000.0)
    return casted_data

# delete unwanted data - just keep date, open, high, low, close
formated_bars = []
for line in bars:
    del line[6:]
    # cast string numbers to integer
    formated_bars.append(list(map(cast_string_to_float, line)))

#  store data in a dataframe
df = pd.DataFrame(formated_bars, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
df.set_index('Date', inplace=True)
# Converting the index as date
df.index = pd.to_datetime(df.index)
df

,Open,High,Low,Close,Volume
Date,,,,,
2021-06-12 01:00:00,2300.19,2309.43,2275.45,2283.46,37902.38468
2021-06-12 02:00:00,2283.46,2301.68,2275.94,2290.36,28211.43025
2021-06-12 03:00:00,2290.34,2302.00,2283.47,2299.11,21840.64096
2021-06-12 04:00:00,2299.11,2325.44,2299.11,2322.12,35648.91872
2021-06-12 05:00:00,2322.12,2322.92,2302.05,2308.01,29285.78369
...,...,...,...,...,...
2021-08-11 23:00:00,3227.53,3239.00,3218.76,3234.61,16009.78884
2021-08-12 00:00:00,3234.62,3234.94,3209.00,3211.95,16079.75244
2021-08-12 01:00:00,3211.96,3219.87,3170.78,3173.66,31596.17127


In [5]:
from talib import EMA
from backtesting.lib import crossover

class TrendFollow(Strategy):
    SLOW_LENGTH = 26
    FAST_LENGTH = 25


    def init(self):
        self.slow_ema = self.I(EMA, self.data.Close, timeperiod=self.SLOW_LENGTH)
        self.fast_ema = self.I(EMA, self.data.Close, timeperiod=self.FAST_LENGTH)

    def next(self):
        if crossover(self.fast_ema, self.slow_ema):
            self.buy()
        elif crossover(self.slow_ema, self.fast_ema):
            self.sell()

In [6]:
bt = Backtest(df, TrendFollow, cash=1000000, commission=.0004, exclusive_orders=True, margin=0.5)
stats = bt.run()
stats

Start                     2021-06-12 01:00:00
End                       2021-08-12 03:00:00
Duration                     61 days 02:00:00
Exposure Time [%]                   97.205181
Equity Final [$]               1691717.664884
Equity Peak [$]                2272304.696964
Return [%]                          69.171766
Buy & Hold Return [%]               36.419731
Return (Ann.) [%]                 2108.962538
Volatility (Ann.) [%]               5579.2958
Sharpe Ratio                         0.377998
Sortino Ratio                       23.955388
Calmar Ratio                        59.424794
Max. Drawdown [%]                  -35.489606
Avg. Drawdown [%]                   -5.377679
Max. Drawdown Duration       16 days 12:00:00
Avg. Drawdown Duration        1 days 13:00:00
# Trades                                   29
Win Rate [%]                         41.37931
Best Trade [%]                      20.773681
Worst Trade [%]                     -6.782459
Avg. Trade [%]                    

In [7]:
bt.plot()